In [ ]:
import sys
sys.path.append("../src")
from imperium.dynamics import auv2d
from imperium.segment import Indirect
from imperium.environments.algae import Algae_Farm
import matplotlib.pyplot as plt, numpy as np, pygmo as pg
%matplotlib

In [ ]:
# create an algae farm
farm = Algae_Farm(5, 5, 2, 6, 10, 40, 40)

# compute waypoint sequence
pts = farm.simple_coverage()

In [ ]:
# AUV characteristics
T = 10 # Newtons
m = 10 # kilograms
homo = 0.99 # homotopy parameter

In [ ]:
# boundary conditions
t0 = 0 # seconds
s0 = np.array([farm.dsx, farm.dsy, 0, 0]) # stationary at dock [m, m, m/s, m/s]
sf = s0 # return softly to dock

In [ ]:
# add boundaries to coverage map
np.random.randn(1 + len(s0)*2)

In [ ]:




seg = Indirect("auv2d", [10, 10], [100, 10, 100], [0, -1, -1], [1, 1, 1], True)

# departure state
t0 = 0
s0 = np.array([farm.dsx, farm.dsy, 0, 0])
tf = 100
sf = np.hstack((pts[-1], [0, 0]))
seg.set(0, np.array([farm.dsx, farm.dsy, 0, 0]), 10, sf, np.random.randn(4)*20, [0.999], True)
seg.mismatch()

In [ ]:
# optimisation problem
prob = pg.problem(seg)

# instantiate SNOPT algorithm
algo = pg.ipopt()
algo.set_numeric_option("tol",1e-5)
algo = pg.algorithm(algo)
algo.set_verbosity(1)

In [ ]:
# instantiate population
pop = pg.population(prob, 1)
pop = algo.evolve(pop)

In [ ]:
# show coverage scenario
seg.fitness(pop.champion_x)
fig, ax = plt.subplots(1)
farm.plot(ax)
pts = farm.simple_coverage()
ax.plot(pts.T[0], pts.T[1], "k.")
ax.plot(seg.states[:, 0], seg.states[:, 1], "k.-")
fig.show()

In [ ]:
dynamics.__dict__['auv2d']